In [4]:
# coding: utf-8

from obspy import *
import struct
import os
import numpy as np
import psutil
import time
def unpackAfile(infile):

# == opening Afile ==
    b= os.path.getsize(infile)
    FH = open(infile, 'rb')
    line = FH.read(b)
    fileHeader= struct.unpack("<4s3h6bh6s", line[0:24])
    
    fileLength = fileHeader[3]
    port = fileHeader[10]
    FirstStn = fileHeader[11][0:4].decode('ASCII').rstrip()
    #print(fileHeader)
# =================================Header===================================
    
    portHeader = []
    for i in range(24,port*32,32):
        port_data = struct.unpack("<4s4s3sbh2b4s12b",line[i:i+32])
        portHeader.append(port_data)

# =================================Data===================================    

    dataStartByte = 24+int(port)*32
    dataPoint = 3*int(port)*int(fileLength)*100
    times = int(port)*3*4
    data=[]

    data = struct.unpack("<%di"%dataPoint,line[dataStartByte:dataStartByte+dataPoint*4])

    
    portHeader = np.array(portHeader)    
    data = np.array(data)    
    idata =data.reshape((3,port,fileLength*100),order='F')
    
#== write to obspy Stream --

    sttime = UTCDateTime(fileHeader[1],fileHeader[4],fileHeader[5],fileHeader[6],fileHeader[7],fileHeader[8],fileHeader[2])
    npts = fileHeader[3]*fileHeader[9]
    samp = fileHeader[9]
    afst = Stream()
    
    for stc in range(fileHeader[10]):
        stn = portHeader[stc][0].decode('ASCII').rstrip()
        instrument = portHeader[stc][1].decode('ASCII').rstrip()
        loc = '0'+str(portHeader[stc][6].decode('ASCII'))
        net = str(portHeader[stc][7].decode('ASCII')).rstrip()
        GPS = int(portHeader[stc][3])
        
        # remove GPS unlock or broken station
        if ( GPS == 1 or GPS == 2 ):
            chc = 0
            if instrument == 'FBA':
                chc = 1
            elif instrument == 'SP':
                chc = 4
            elif instrument == 'BB':
                chc = 7
            
            # for each channel in port
            for ch in range(3):
                chn = 'Ch'+str(chc+ch)
                stats = {'network': net, 'station': stn, 'location': loc,
                        'channel': chn, 'npts': npts, 'sampling_rate': samp,
                        'starttime': sttime}
                
                data = np.array(idata[ch][stc], dtype=float)
                sttmp = Stream([Trace(data=data, header=stats)])
                afst += sttmp
    
    return afst, FirstStn, fileHeader
      
data, FirstStnm, fileHeader= unpackAfile('./05290646.40C')

import pymysql
#連接到mysql
def connect2mysql(host,user,passwd,db):
    db = pymysql.connect(
        host=host,
        port=3306,
        charset='utf8',
        user=user,
        passwd=passwd,
        db=db
    )
    cursor = db.cursor()
    return db, cursor

#將資料存入mysql
def insert2mysql(table, data):
    sql = """INSERT INTO """+table+"""(network,station,location,channel,starttime,endtime,sampling_rate,delta,npts,calib,data) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    l = len(data)
    inserts = []
    for i in range(0,l):
        inserts.append([str(data[i].stats['network']), str(data[i].stats['station']), 
        str(data[i].stats['location']), str(data[i].stats['channel']), str(data[i].stats['starttime']),
        str(data[i].stats['endtime']), str(data[i].stats['sampling_rate']), str(data[i].stats['delta']), 
        str(data[i].stats['npts']), str(data[i].stats['calib']), str(data[i].data)])
    cpu=psutil.cpu_percent()
    memory=psutil.virtual_memory()
    memory=memory.available
    memory=memory/1024
    memory=memory/1024
    memory=memory/1024
    cursor.executemany(sql,inserts)
    db.commit()
    return cpu, memory

#查找Table內所有資料
def selectAll(table):
    cursor.execute("select * from "+table+" limit 500")
    data = cursor.fetchall()
    return data

#查找指定station之資料
def selectStation(table, station):
    cursor.execute("select * from "+table+" where station='%s'" % (station))
    data = cursor.fetchall()
    return data

import datetime
#查找指定時間段內之資料
def selectDateRange(table,starttime,endtime):
    cursor.execute("select * from "+table+" where starttime >'%s' and endtime < '%s'" % (starttime,endtime))
    data = cursor.fetchall()
    return data
	
#以station來分類得到各類資料筆數
def selectGroupByStation(table, network):
    cursor.execute("select station,count(*) from "+table+" where network ='%s' group by station" % (network))
    data = cursor.fetchall()
    return data

#得到指定時間段內之資料筆數
def selectDateRangeCount(table,starttime,endtime):
    rowCount = cursor.execute("select * from "+table+" where starttime >'%s' and endtime < '%s'" % (starttime,endtime))
    return rowCount

import time
#計算mysql操作所花費的時間
def timeSpent(f):
    def wrapper(*args, **kwargs):
        start=time.time()
        response = f(*args, **kwargs)
        #print ('Time:', time.time()-start)
        return response
    return wrapper

#配合上面之api來計算mysql操作時間
@timeSpent
def insertTest(times,data,table):
    cpu_temp=[]
    memory_temp=[]
    for i in range(times):
        temp1, temp2=insert2mysql(table,data)
        cpu_temp.append(temp1)
        memory_temp.append(temp2)
    return cpu_temp, memory_temp
    
@timeSpent
def selectAllQuery(table):
    start=time.time()
    selectAll(table)
    end = time.time()
    end = end-start
    end = round(end,4)
    return end
  
@timeSpent
def selectStationQuery(table,station='TAP'):
    selectStation(table,station)
    return
  
@timeSpent
def selectDateRangeQuery(table):
    start = datetime.datetime(2018, 5, 24, 7, 51, 4)
    end = datetime.datetime(2018, 9, 24, 7, 52, 4)
    selectDateRange(table,start,end)
    return

@timeSpent
def selectDateRangeCountQuery(table):
    start = datetime.datetime(2018, 5, 24, 7, 51, 4)
    end = datetime.datetime(2018, 9, 24, 7, 52, 4)
    selectDateRangeCount(table,start,end)
    return
  
@timeSpent        
def selectGroupByStationQuery(table, network='SMT'):
    selectGroupByStation(table, network)
    return

def showtables():
    cursor.execute(" show tables ")
    tables = cursor.fetchall()
    return tables

def averagecpuusage(cpu_usage):
    total_cpu = sum(cpu_usage)
    total_cpu= total_cpu/len(cpu_usage)
    total_cpu=round(total_cpu)
    return total_cpu
def averagememoryusage(memory_usage):
    total_memory = sum(memory_usage)
    total_memory = total_memory/len(memory_usage)
    total_memory=round(total_memory,2)
    return total_memory

#效能測試1
db, cursor=connect2mysql('127.0.0.1','root','v09716432','test') #連線
ExistTable = showtables()
# 20W/36 = 5556 資料20萬筆 case 1
# 5W/36 = 1390 資料5萬筆 case 2

caseInfo = ['20萬筆資料','5萬筆資料']
caseInfo1 = ['200000','50000']
databaseCollection = [{'database':'test','collection':'test1', 'Loops':5556},
                      {'database':'test','collection':'test2', 'Loops':1390}
                     ]

usage_time=[]
cpu_usage_percent=[]
memmory_usage=[]
select_time=[]

for i in range(2):
  time_a = time.time()
  print('Case: ', i+1, ' ', caseInfo[i])
  cpu1, memory1 = insertTest(databaseCollection[i]['Loops'], data, databaseCollection[i]['collection'])
  #print(cpu1)
  #print(memory1)
  selectAllQuery(databaseCollection[i]['collection'])
  time_select = selectAllQuery(databaseCollection[i]['collection'])
  print(time_select)
  select_time.append(time_select)
  selectStationQuery(databaseCollection[i]['collection'])
  selectDateRangeQuery(databaseCollection[i]['collection'])
  selectDateRangeCountQuery(databaseCollection[i]['collection'])
  selectGroupByStationQuery(databaseCollection[i]['collection'])
  time_b = time.time()
  print("CPU used in %d seconds: %s Percent" % (time_b-time_a,averagecpuusage(cpu1)))
  print("Memory used in %d seconds: %s GB" % (time_b-time_a,averagememoryusage(memory1)))
  usage_time.append(round(time_b-time_a,2))
  cpu_usage_percent.append(averagecpuusage(cpu1))
  memmory_usage.append(averagememoryusage(memory1))
    
import csv
import codecs
Database = dict()
with open ('mysql_usage1.csv', 'w', newline = '', encoding='big5') as csvfile:
    datanames=['CaseCount', 'Usage_time', 'CpuCount', 'Cpu_percent', 'Cpu_usage', 'Memory_usage', 'Select_AllData_time']
    writer = csv.DictWriter(csvfile , fieldnames= datanames)
    writer.writeheader()
    for i in range(0,len(usage_time)):
        Database['CaseCount']=caseInfo1[i]
        Database['Usage_time']=usage_time[i]
        Database['CpuCount']=psutil.cpu_count()
        Database['Cpu_percent']=cpu_usage_percent[i]
        c_count=psutil.cpu_count()
        temp=cpu_usage_percent[i]
        temp=temp/100
        c_count=c_count*temp
        c_count=round(c_count,2)
        Database['Cpu_usage']=c_count
        Database['Memory_usage']=memmory_usage[i]
        Database['Select_AllData_time']=select_time[i]
        writer.writerow(Database)
'''
selectAllQuery('test1')
selectAllQuery('test2')
selectAllQuery('test3')
selectAllQuery('test4')
selectAllQuery('test5')
'''
db.close() #關閉連接

Case:  1   20萬筆資料
0.0175
CPU used in 1078 seconds: 26 Percent
Memory used in 1078 seconds: 2.62 GB
Case:  2   5萬筆資料
0.013
CPU used in 260 seconds: 26 Percent
Memory used in 260 seconds: 2.15 GB
